In [ ]:


def order_time(ele): #排序的KEY
    return ele['orderdate']
def order_ship(ele):
    return ele['ship']

def prdinfo(findtitle,text): #分析商品資訊
    prdtext = text.find(findtitle)#找到數量的位置
    firstsplit = prdtext+len(findtitle)+text[prdtext+len(findtitle):len(text)].find(";") #找到數量之後第一個分號位置
    
    if prdtext == -1:
        return ""
    return text[prdtext+len(findtitle):firstsplit].strip().replace(",","")#需要的片段
def isodetect(iso):
    a = 0
    for i in isoAry:
        a += 1
        if i[0].find(iso) != -1:
            if i[0] == "": #如果找到的ISO碼是空的 則返回FALSE
                return False
            return i
        
    return False
def browser():
    fname = filedialog.askopenfilename(initialdir= os.getcwd(),filetypes = (("Excel","*.xls"),("all files","*.*")))
    global filepath
    filepath =  fname  # 返回文件全路径
    pathinput.delete('0',tk.END)
    pathinput.insert(0,filepath)
    
gui=tk.Tk()
gui.title("蝦皮出貨單")
gui.geometry("450x50")
pathinput = tk.Entry(width=50)
pathinput.pack(side="left")
btn2 = tk.Button(gui, text ="瀏覽", command = browser)
btn2.pack(side="left")
btn = tk.Button(gui, text ="執行", command = run)
btn.pack(side="left")
gui.mainloop()


In [ ]:
#蝦皮庫存表整理 新版
import xlrd
import xlwt
import re
import time
import requests
from bs4 import BeautifulSoup
import tkinter as tk
import tkinter.filedialog as filedialog
import os
#時間
def run():
    today_str = time.strftime("%Y",time.localtime())+"/"+str(int(time.strftime("%m",time.localtime())))+"/"+str(int(time.strftime("%d",time.localtime())))
    #ISO 資料
    url =  "https://docs.google.com/spreadsheets/d/19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI/gviz/tq?tqx=out:html&tq=select%20B,C,D&gid=0"
    gotrq = requests.get(url)
    soup = BeautifulSoup(gotrq.text, "html.parser")
    tr = soup.select("tr")
    global isoAry
    isoAry = [] #獲取網站上的ISO 資料
    for r in tr:    
        try:
            isoAry.append([r.td.text,r.td.next_sibling.text,r.td.next_sibling.next_sibling.text])
        except:
            print('notd')

    #讀取XLS
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(filepath)
    table1 = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table1.nrows #取总行数


    #商品資訊 陣列
    orderObj = []
    nowOrderID = '00000'
    obj_count = 0
    for i in range(1,rows_count): #迴圈
        prd_i = i
        #如果第orderID 不一樣寫入資料
        if table1.cell_value(i,0) != nowOrderID:
            obj_count += 1
            orderObj.append({'note':"",'orderdate':"",'orderID':"",'name':"",'tel':"",'account':"",'shipPrice':"",'ship':"",'shipID':"",'orderPrice':"",'discount':"",'credit':"",'prds':[]})#新增新的物件
            nowOrderID = table1.cell_value(i,0) 
            orderObj[obj_count-1]['orderID'] = table1.cell_value(i,0) 
            orderObj[obj_count-1]['orderdate'] = table1.cell_value(i,4) 
            orderObj[obj_count-1]['name'] = table1.cell_value(i,33) 
            orderObj[obj_count-1]['tel'] = table1.cell_value(i,28) 
            orderObj[obj_count-1]['account'] = table1.cell_value(i,3) 
            orderObj[obj_count-1]['shipPrice'] = table1.cell_value(i,6) 
            orderObj[obj_count-1]['ship'] = table1.cell_value(i,34) 
            orderObj[obj_count-1]['shipID'] = table1.cell_value(i,40) 
            orderObj[obj_count-1]['orderPrice'] = table1.cell_value(i,7) 
            orderObj[obj_count-1]['discount'] = table1.cell_value(i,12) 
            orderObj[obj_count-1]['note'] = table1.cell_value(i,44)+" , "+table1.cell_value(i,45)
            if table1.cell_value(i,38) == "信用卡":
                orderObj[obj_count-1]['credit'] = table1.cell_value(i,38)

            for ii in range(prd_i,rows_count): #商品資料
                orgtext = table1.cell_value(ii,18) 
                prd_price = table1.cell_value(ii,20) 
                if table1.cell_value(ii,21):  #如果特價有值，取代原價
                    prd_price = table1.cell_value(ii,21) 

                prd_iso = table1.cell_value(ii,22) 
                if table1.cell_value(ii,23):  #如果款式ISO有值，取代
                    prd_iso = table1.cell_value(ii,23) 

                orderObj[obj_count-1]['prds'].append({'prdText':orgtext,'count':table1.cell_value(ii,24) ,'price':prd_price,'iso':prd_iso.replace(" ", "").replace(" ", "").replace("	","")}) #放入prd陣列中，-1是因為i是從第二列開始
                    
                try:
                    if nowOrderID != table1.cell_value(ii+1,0): #如果下一列訂單貨號不一樣跳出
                        i = ii
                        break
                except:
                    break
            if float(orderObj[obj_count-1]['discount']):
                orderObj[obj_count-1]['prds'].append({'prdText':'折扣碼','count':"1",'price': float(orderObj[obj_count-1]['discount'])*-1,'iso':"折扣碼"}) 



    #排序訂單
    orderObj.sort(key=order_time,reverse=True)
    orderObj.sort(key=order_ship)

    
    #寫XLS
    nwb = xlwt.Workbook()# 新增EXCEL
    nsheet = nwb.add_sheet('google試算表格式')#新SHEET
    nsheet.write(0,0,'日期')
    nsheet.write(0,1,'訂單編號')
    nsheet.write(0,2,'姓名')
    nsheet.write(0,3,'電話')
    nsheet.write(0,4,'帳號')
    nsheet.write(0,5,'國際條碼')
    nsheet.write(0,6,'產品名稱')
    nsheet.write(0,7,'產品款式')
    nsheet.write(0,8,'數量')
    nsheet.write(0,9,'價格')
    nsheet.write(0,10,'總價')
    nsheet.write(0,11,'貨運')
    nsheet.write(0,12,'運費')
    nsheet.write(0,13,'發票')
    nsheet.write(0,16,'付款方式')
    nsheet.write(0,21,'折扣')
    nsheet.write(0,22,'備註')
    write_row = 1

    for i in orderObj:
        orderstart = 0
        for ii in i['prds']:
            orderstart += 1
            if orderstart == 1:
                nsheet.write(write_row,12,int(float(i['shipPrice'])))
                nsheet.write(write_row,11,i['ship'])
                nsheet.write(write_row,22,i['note'])
                nsheet.write(write_row,13,int(float(i['orderPrice'])))#扣掉折扣 這是訂單上的折扣(正)，非商品資訊內折扣(負號)
                if i['credit'] == '信用卡':
                    nsheet.write(write_row,16,i['credit'])
                if float(i['discount']):
                    nsheet.write(write_row,21,int(float(i['discount'])))
            nsheet.write(write_row,0,today_str)
            nsheet.write(write_row,1,i['orderID'])
            nsheet.write(write_row,2,i['name']+i['shipID'][-3:])
            nsheet.write(write_row,3,i['tel'])
            nsheet.write(write_row,4,i['account'])
            
            #商品資料
            iso_data = isodetect(ii['iso'])
            if iso_data:
                nsheet.write(write_row,5,iso_data[0]) #ISO
                nsheet.write(write_row,6,iso_data[1]) #產品名
                nsheet.write(write_row,7,iso_data[2]) #款式
            else: #找不到ISO
                if ii['prdText'][6:ii['prdText'].find(" 價格:")] == '': #如果是折扣碼
                    print(write_row,ii['prdText'])
                    nsheet.write(write_row,6,ii['prdText'])
                else: #非折扣碼
                    nsheet.write(write_row,6,ii['prdText'][6:ii['prdText'].find(" 價格:")])
                    


            ct = ii['count']
            pe = ii['price']
            nsheet.write(write_row,8,ct)
            nsheet.write(write_row,9,int(float(pe)))
            nsheet.write(write_row,10,int(float(ct)*float(pe)))

            write_row += 1

    nwb.save('result.xls')

    


In [ ]:
text2 = "[1]a[1]a[1]a[21]a[1]a[9]a[10]"
re.split("\[[0-9][0-9]*\]",text2) #分割"[數字]"資料

In [ ]:
a = [{'note': ' , ', 'orderdate': '2019-03-28 08:48', 'orderID': '190328084766HPS', 'name': '連*婷', 'tel': '886929***389', 'account': 'ceheer', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14545888700', 'orderPrice': '930.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜 撈魚網 不破網 10支入 8支入 5支入 3支入 🌳 綠光森林 🌳; 商品選項名稱 (品):10支入; 價格: $ 190; 數量: 2; 商品選項貨號: 9022200300101x10;', 'count': '2', 'price': '190', 'iso': '9022200300101x10'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 日本進口  【縁日】撈金魚 重複式網架(5入) / 替換網紙(100入)  撈魚網  可換網紙; 商品選項名稱 (品):重複式網架(5入); 價格: $ 90; 數量: 5; 商品選項貨號: 4905842363022x5;', 'count': '5', 'price': '90', 'iso': '4905842363022x5'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 日本進口  【縁日】撈金魚 重複式網架(5入) / 替換網紙(100入)  撈魚網  可換網紙; 商品選項名稱 (品):替換網紙(100入); 價格: $ 40; 數量: 1; 商品選項貨號: 4905842390110x100;', 'count': '1', 'price': '40', 'iso': '4905842390110x100'}]}, {'note': '出貨煩請檢查商品及包裝有無瑕疵 , ', 'orderdate': '2019-03-28 01:04', 'orderID': '19032801036RGEN', 'name': 'K*i', 'tel': '886909***015', 'account': 'kai800115', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14545876277', 'orderPrice': '235.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):✨現貨快出🔜【John's Blend】香氛噴霧280ml 麝香櫻花 🌳 綠光森林 🌳; 商品選項名稱 (品):香氛噴霧280ml (超商限15瓶); 價格: $ 175; 數量: 1; 商品選項貨號: 4535304291619;", 'count': '1', 'price': '175', 'iso': '4535304291619'}]}, {'note': ' , ', 'orderdate': '2019-03-27 23:39', 'orderID': '19032723386H5DS', 'name': '黃*琳', 'tel': '886976***215', 'account': 'st970167', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14545893417', 'orderPrice': '185.00', 'discount': '0.00', 'credit': '信用卡', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜【LION獅王】衣物局部去漬劑160g 🌳 綠光森林 🌳; 價格: $ 125; 數量: 1; 主商品貨號: 4903301527039;', 'count': '1', 'price': '125', 'iso': '4903301527039'}]}, {'note': ' , ', 'orderdate': '2019-03-27 23:02', 'orderID': '19032723016CJMT', 'name': '陳*荏', 'tel': '886955***050', 'account': 'a0936882760', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546163742', 'orderPrice': '800.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜日本 柳屋YANAGIYA HAIR TONIC 髮根 養髮液 清爽 柑橘 240ML 🌳綠光森林🌳; 商品選項名稱 (品):髮根養髮液240ml; 價格: $ 245; 數量: 1; 商品選項貨號: 4903018113235;', 'count': '1', 'price': '245', 'iso': '4903018113235'}, {'prdText': '商品名稱 (品):✨現貨快出🔜日本 柳屋YANAGIYA HAIR TONIC 髮根 養髮液 清爽 柑橘 240ML 🌳綠光森林🌳; 商品選項名稱 (品):髮根養髮液240ml 柑橘; 價格: $ 245; 數量: 1; 商品選項貨號: 4903018113518;', 'count': '1', 'price': '245', 'iso': '4903018113518'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 SHISEIDO 資生堂 爽快秀髮乾洗髮150ml／附噴頭 🌳 綠光森林 🌳; 價格: $ 130; 數量: 1; 主商品貨號: 4901872841974;', 'count': '1', 'price': '130', 'iso': '4901872841974'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304275893;", 'count': '1', 'price': '120', 'iso': '4535304275893'}]}, {'note': ' , ', 'orderdate': '2019-03-27 21:32', 'orderID': '190327213061B58', 'name': '林*諺', 'tel': '886975***788', 'account': 'yan_1107', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546176900', 'orderPrice': '516.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):💯㊣附發票🔜日本John's Blend 香氛片 白麝香 紅酒 茉莉 香片 衣櫥芳香 房間室內 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香; 價格: $ 48; 數量: 2; 商品選項貨號: 4535304275800;", 'count': '2', 'price': '48', 'iso': '4535304275800'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜  John's Blend 香氛噴霧280ml 白麝香 蘋果梨 白茉莉 紅酒香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 宅配限15個; 價格: $ 175; 數量: 1; 商品選項貨號: 4535304275831;", 'count': '1', 'price': '175', 'iso': '4535304275831'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 熊野 麗白薏仁化妝水 500ml 1000ml  🌳 綠光森林 🌳; 商品選項名稱 (品):1000ml; 價格: $ 185; 數量: 1; 商品選項貨號: 4513574025875;', 'count': '1', 'price': '185', 'iso': '4513574025875'}]}, {'note': ' , ', 'orderdate': '2019-03-27 17:49', 'orderID': '19032717486BQ75', 'name': '許*燊', 'tel': '886989***789', 'account': 'dirlogue', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546185739', 'orderPrice': '344.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票 🔜 ECHO 電動攪拌器  🌳 綠光森林 🌳; 價格: $ 50; 數量: 3; 主商品貨號: 4991203130631;', 'count': '3', 'price': '50', 'iso': '4991203130631'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【Daiya】肥皂瀝水盤 粉色 白色 棕色 🌳 綠光森林 🌳; 商品選項名稱 (品):粉色; 價格: $ 95; 數量: 1; 商品選項貨號: 4901948615102;', 'count': '1', 'price': '95', 'iso': '4901948615102'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【INOMATA】牛奶鍋造型量杓 白色 橘藍 🌳 綠光森林 🌳; 商品選項名稱 (品):橘藍; 價格: $ 39; 數量: 1; 商品選項貨號: 4905596113799;', 'count': '1', 'price': '39', 'iso': '4905596113799'}]}, {'note': ' , ', 'orderdate': '2019-03-27 16:50', 'orderID': '190327164966D9F', 'name': '包*雄', 'tel': '886980***825', 'account': 'may7993590', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546303761', 'orderPrice': '825.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票 🔜 SHISEIDO 資生堂 ECO SOAP 泡泡洗顏皂 18g 🌳 綠光森林 🌳; 商品選項名稱 (品):洗顏皂(金色包裝); 價格: $ 20; 數量: 1; 商品選項貨號: JL000001;', 'count': '1', 'price': '20', 'iso': 'JL000001'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 Loshi 馬油身體乳液485ml 保濕乳液 🌳 綠光森林 🌳; 商品選項名稱 (品):1罐; 價格: $ 140; 數量: 1; 商品選項貨號: 4936201102297;', 'count': '1', 'price': '140', 'iso': '4936201102297'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 小久保 伸縮式洗瓶刷 🌳 綠光森林 🌳; 價格: $ 50; 數量: 1; 主商品貨號: 4956810235759;', 'count': '1', 'price': '50', 'iso': '4956810235759'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜熊野 salon link沙龍級洗護兩效合一洗髮精1000ml  🌳 綠光森林 🌳; 商品選項名稱 (品):洗護兩效合一洗髮精(4瓶以上限宅配); 價格: $ 159; 數量: 1; 商品選項貨號: 4513574022812;', 'count': '1', 'price': '159', 'iso': '4513574022812'}, {'prdText': '商品名稱 (品):✨現貨快出【雞仔牌】部屋用果凍芳香劑260g 黑紫色 純真紫丁香 黑紅色 神秘玫瑰香 金色 浪漫芙蓉香🌳綠光森林🌳; 商品選項名稱 (品):黑紫色&純真紫丁香; 價格: $ 135; 數量: 1; 商品選項貨號: 125852;', 'count': '1', 'price': '135', 'iso': '125852'}, {'prdText': '商品名稱 (品):✨現貨快出【雞仔牌】部屋用果凍芳香劑260g 黑紫色 純真紫丁香 黑紅色 神秘玫瑰香 金色 浪漫芙蓉香🌳綠光森林🌳; 商品選項名稱 (品):金色&浪漫芙蓉香; 價格: $ 135; 數量: 1; 商品選項貨號: 126538;', 'count': '1', 'price': '135', 'iso': '126538'}, {'prdText': '商品名稱 (品):✨現貨快出🔜日本製 SANADA 不動化學 布鞋強力去污棒 洗鞋棒🌳 綠光森林 🌳; 商品選項名稱 (品):強力去污棒／布鞋用100g; 價格: $ 39; 數量: 1; 商品選項貨號: 4984324007265;', 'count': '1', 'price': '39', 'iso': '4984324007265'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜CLOVER 薏仁種子洗顏皂125g×2入 🌳 綠光森林 🌳; 商品選項名稱 (品):薏仁種子洗顏皂125g×2入; 價格: $ 87; 數量: 1; 商品選項貨號: 4901498116678;', 'count': '1', 'price': '87', 'iso': '4901498116678'}]}, {'note': ' , ', 'orderdate': '2019-03-27 13:12', 'orderID': '19032713116E1RV', 'name': '張*詠', 'tel': '886910***271', 'account': 'jhu142611', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546311942', 'orderPrice': '1263.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜 【Sanko】免洗劑排水口清潔菜瓜布刷 綠色 🌳 綠光森林 🌳; 價格: $ 110; 數量: 1; 主商品貨號: 4973381250066;', 'count': '1', 'price': '110', 'iso': '4973381250066'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【伊勢藤】伸縮式水桶3.7L 🌳 綠光森林 🌳; 商品選項名稱 (品):綠藍3.7L; 價格: $ 390; 數量: 1; 商品選項貨號: 536120;', 'count': '1', 'price': '390', 'iso': '536120'}, {'prdText': '商品名稱 (品):✨現貨快出🔜 【KAO花王】高黏度衛浴&廚房水管清潔凝膠500g 🌳 綠光森林 🌳; 價格: $ 99; 數量: 1; 主商品貨號: 4901301307453;', 'count': '1', 'price': '99', 'iso': '4901301307453'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【MAM】保溫瓶罐清洗刷具組3支入  🌳 綠光森林 🌳; 價格: $ 46; 數量: 1; 主商品貨號: 4930419480789;', 'count': '1', 'price': '46', 'iso': '4930419480789'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜小久保 排水口刷 白色 🌳 綠光森林 🌳; 價格: $ 49; 數量: 1; 主商品貨號: 4956810803163;', 'count': '1', 'price': '49', 'iso': '4956810803163'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜創和 免洗劑海綿菜瓜布 🌳 綠光森林 🌳; 價格: $ 65; 數量: 1; 主商品貨號: 4944201001793;', 'count': '1', 'price': '65', 'iso': '4944201001793'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜雞仔牌 洗衣槽除菌劑550g 2入 單瓶 🌳 綠光森林 🌳; 商品選項名稱 (品):洗衣槽除菌劑550g 2入(超商限4組); 價格: $ 120; 數量: 1; 商品選項貨號: 4901070939145;', 'count': '1', 'price': '120', 'iso': '4901070939145'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 小久保 免洗劑食器清潔菜瓜布-橘色 🌳 綠光森林 🌳; 價格: $ 125; 數量: 1; 主商品貨號: 4956810232963;', 'count': '1', 'price': '125', 'iso': '4956810232963'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜AIMEDIA 超強白效布鞋洗滌劑100g 🌳 綠光森林; 商品選項名稱 (品):超強白效布鞋洗滌劑100g; 價格: $ 199; 數量: 1; 商品選項貨號: 4989409076570;', 'count': '1', 'price': '199', 'iso': '4989409076570'}]}, {'note': ' , ', 'orderdate': '2019-03-27 11:14', 'orderID': '19032711106X61N', 'name': '劉*諒', 'tel': '886978***638', 'account': 'oocc1972', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546282683', 'orderPrice': '580.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):蘋果梨 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304282884;", 'count': '1', 'price': '120', 'iso': '4535304282884'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白茉莉 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304287681;", 'count': '1', 'price': '120', 'iso': '4535304287681'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):麝香櫻花 (超取限制12瓶); 價格: $ 140; 數量: 2; 商品選項貨號: 4535304291626;", 'count': '2', 'price': '140', 'iso': '4535304291626'}]}, {'note': ' , ', 'orderdate': '2019-03-27 10:44', 'orderID': '19032710426U3CF', 'name': '王*惠', 'tel': '886988***033', 'account': 'kiki1016', 'shipPrice': '60.00', 'ship': '7-11', 'shipID': 'F14546372610', 'orderPrice': '294.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票 🔜  日本 正品 東邦 去污垢魔法皂133g 🌳 綠光森林 🌳; 商品選項名稱 (品):魔法皂 (超商限36顆); 價格: $ 39; 數量: 6; 主商品貨號: 4904766110309;', 'count': '6', 'price': '39', 'iso': '4904766110309'}]}, {'note': ' , ', 'orderdate': '2019-03-28 03:56', 'orderID': '19032803546W64U', 'name': '黃*婷', 'tel': '886919***651', 'account': 'mei6146', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '70749687762', 'orderPrice': '410.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304275893;", 'count': '1', 'price': '120', 'iso': '4535304275893'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白茉莉 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304287681;", 'count': '1', 'price': '120', 'iso': '4535304287681'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):麝香櫻花 (超取限制12瓶); 價格: $ 140; 數量: 1; 商品選項貨號: 4535304291626;", 'count': '1', 'price': '140', 'iso': '4535304291626'}]}, {'note': ' , ', 'orderdate': '2019-03-28 02:23', 'orderID': '19032802226UPDK', 'name': '謝*芩', 'tel': '886919***685', 'account': 'silver427t', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '27005265080', 'orderPrice': '1221.00', 'discount': '0.00', 'credit': '信用卡', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票 🔜  PONIKASEI 伸縮衣架1入 粉色 藍色 隨機出貨 🌳 綠光森林 🌳; 價格: $ 49; 數量: 4; 主商品貨號: 4962964105107;', 'count': '4', 'price': '49', 'iso': '4962964105107'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 熊野 10種植物萃取精華洗髮精 潤髮乳 700ml 🌳 綠光森林 🌳; 商品選項名稱 (品):洗髮精+潤髮乳 組合價; 價格: $ 390; 數量: 1; 商品選項貨號: t0029;', 'count': '1', 'price': '390', 'iso': 't0029'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜小林製藥 鞋用除菌消臭噴霧250ml 🌳 綠光森林 🌳; 價格: $ 135; 數量: 1; 主商品貨號: 4987072061756;', 'count': '1', 'price': '135', 'iso': '4987072061756'}, {'prdText': '商品名稱 (品):✨現貨快出🔜 【KAO花王】高黏度衛浴&廚房水管清潔凝膠500g 🌳 綠光森林 🌳; 價格: $ 99; 數量: 1; 主商品貨號: 4901301307453;', 'count': '1', 'price': '99', 'iso': '4901301307453'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【SANADA】強力去污棒／衣類專用100g 日本製 🌳 綠光森林 🌳; 商品選項名稱 (品):強力去污棒／衣類專用100g; 價格: $ 39; 數量: 1; 商品選項貨號: 4984324007272;', 'count': '1', 'price': '39', 'iso': '4984324007272'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【Pelican】泥炭石洗顏石鹼皂100g 🌳 綠光森林 🌳; 商品選項名稱 (品):【Pelican】泥炭石洗顏石鹼皂100; 價格: $ 147; 數量: 1; 商品選項貨號: 4976631649191; 主商品貨號: 4976631649191;', 'count': '1', 'price': '147', 'iso': '49766316491914976631649191'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 熊野 麗白薏仁化妝水 500ml 1000ml  🌳 綠光森林 🌳; 商品選項名稱 (品):1000ml; 價格: $ 185; 數量: 1; 商品選項貨號: 4513574025875;', 'count': '1', 'price': '185', 'iso': '4513574025875'}]}, {'note': ' , ', 'orderdate': '2019-03-27 16:12', 'orderID': '190327161062Q6B', 'name': '鍾*寶', 'tel': '886919***416', 'account': 'winduptiff', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '69261772933', 'orderPrice': '390.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 (超取限制12瓶); 價格: $ 120; 數量: 3; 商品選項貨號: 4535304275893;", 'count': '3', 'price': '120', 'iso': '4535304275893'}]}, {'note': ' , ', 'orderdate': '2019-03-27 13:16', 'orderID': '19032713156EFSA', 'name': '林*虹', 'tel': '886911***794', 'account': 'yiingtulip', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '09733567898', 'orderPrice': '410.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票  SUNSTAR三詩達 兒童巧虎牙刷6個月 2~4歲 4~6歲 6~12歲  隨機出貨🌳 綠光森林 🌳; 商品選項名稱 (品):2~4歲; 價格: $ 38; 數量: 10; 商品選項貨號: 4901616214835;', 'count': '10', 'price': '38', 'iso': '4901616214835'}]}, {'note': ' , ', 'orderdate': '2019-03-27 12:44', 'orderID': '19032712426AC5D', 'name': '江*宜', 'tel': '886937***224', 'account': 'bear700224', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '96676930920', 'orderPrice': '908.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票 🔜 SUNSTAR 三詩達 巧虎兒童牙膏70g 草莓 葡萄 日本製  🌳 綠光森林 🌳; 商品選項名稱 (品):葡萄; 價格: $ 55; 數量: 1; 商品選項貨號: 4901616009516;', 'count': '1', 'price': '55', 'iso': '4901616009516'}, {'prdText': '商品名稱 (品):💯㊣附發票  SUNSTAR三詩達 兒童巧虎牙刷6個月 2~4歲 4~6歲 6~12歲  隨機出貨🌳 綠光森林 🌳; 商品選項名稱 (品):4~6歲; 價格: $ 38; 數量: 1; 商品選項貨號: 4901616214828;', 'count': '1', 'price': '38', 'iso': '4901616214828'}, {'prdText': "商品名稱 (品):💯㊣附發票🔜日本John's Blend 香氛片 白麝香 紅酒 茉莉 香片 衣櫥芳香 房間室內 🌳 綠光森林 🌳; 商品選項名稱 (品):蘋果梨; 價格: $ 48; 數量: 2; 商品選項貨號: 4535304282860;", 'count': '2', 'price': '48', 'iso': '4535304282860'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【SUNLIFE】彩色耐熱八角筷子五雙入 🌳 綠光森林 🌳; 價格: $ 140; 數量: 1; 主商品貨號: 4974263317594;', 'count': '1', 'price': '140', 'iso': '4974263317594'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜SUNLIFE 彩色耐熱六角筷子五雙入 🌳 綠光森林 🌳; 價格: $ 140; 數量: 1; 主商品貨號: 4974263316450;', 'count': '1', 'price': '140', 'iso': '4974263316450'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【興洋】耐熱六角筷子五雙入 🌳 綠光森林 🌳; 價格: $ 160; 數量: 1; 主商品貨號: 4901029110052 ;', 'count': '1', 'price': '160', 'iso': '4901029110052'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):蘋果梨 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304282884;", 'count': '1', 'price': '120', 'iso': '4535304282884'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【SANADA】薄型冰箱除臭劑 冷凍室用 野菜室用 冷藏室用 🌳 綠光森林 🌳; 商品選項名稱 (品):冷凍室用; 價格: $ 45; 數量: 1; 商品選項貨號: 4984324008163;', 'count': '1', 'price': '45', 'iso': '4984324008163'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【SANADA】薄型冰箱除臭劑 冷凍室用 野菜室用 冷藏室用 🌳 綠光森林 🌳; 商品選項名稱 (品):冷藏室用; 價格: $ 45; 數量: 1; 商品選項貨號: 4984324008095;', 'count': '1', 'price': '45', 'iso': '4984324008095'}, {'prdText': '商品名稱 (品):✨現貨快出🔜 【INOMATA】冰箱迷你收納架2入組 🌳 綠光森林 🌳; 價格: $ 39; 數量: 1; 主商品貨號: 4905596035800;', 'count': '1', 'price': '39', 'iso': '4905596035800'}]}, {'note': ' , ', 'orderdate': '2019-03-27 09:49', 'orderID': '19032709496NGR8', 'name': '林*如', 'tel': '886928***025', 'account': 'marylin1234', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '47839907651', 'orderPrice': '518.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票  SUNSTAR三詩達 兒童巧虎牙刷6個月 2~4歲 4~6歲 6~12歲  隨機出貨🌳 綠光森林 🌳; 商品選項名稱 (品):4~6歲; 價格: $ 38; 數量: 3; 商品選項貨號: 4901616214828;', 'count': '3', 'price': '38', 'iso': '4901616214828'}, {'prdText': '商品名稱 (品):💯㊣附發票  SUNSTAR三詩達 兒童巧虎牙刷6個月 2~4歲 4~6歲 6~12歲  隨機出貨🌳 綠光森林 🌳; 商品選項名稱 (品):6~12歲; 價格: $ 38; 數量: 3; 商品選項貨號: 4901616214811;', 'count': '3', 'price': '38', 'iso': '4901616214811'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 N47 紗窗刷 日本 Nippon Seal 乾濕二用 紗窗紗門 清潔刷 🌳 綠光森林 🌳; 價格: $ 185; 數量: 1; 主商品貨號: 4902391047663 ;', 'count': '1', 'price': '185', 'iso': '4902391047663'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 LEC 激落君排水孔清潔錠30g×2入 🌳 綠光森林 🌳; 商品選項名稱 (品):激落君排水孔清潔錠30g×2入; 價格: $ 75; 數量: 1; 商品選項貨號: 4903320555785;', 'count': '1', 'price': '75', 'iso': '4903320555785'}]}, {'note': ' , ', 'orderdate': '2019-03-27 09:20', 'orderID': '19032709206JSTY', 'name': '毛*偉', 'tel': '886932***273', 'account': 'wg7780', 'shipPrice': '30.00', 'ship': 'OK Mart', 'shipID': '10646918108', 'orderPrice': '765.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):💯㊣附發票 🔜OSK 可橫放冷水壺2.2L／深藍色  🌳 綠光森林 🌳; 價格: $ 225; 數量: 1; 主商品貨號: 4970825081804 ;', 'count': '1', 'price': '225', 'iso': '4970825081804'}, {'prdText': '商品名稱 (品):✨現貨快出🔜【SUNLIFE】彩色耐熱八角筷子五雙入 🌳 綠光森林 🌳; 價格: $ 140; 數量: 1; 主商品貨號: 4974263317594;', 'count': '1', 'price': '140', 'iso': '4974263317594'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304275893;", 'count': '1', 'price': '120', 'iso': '4535304275893'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):麝香櫻花 (超取限制12瓶); 價格: $ 140; 數量: 1; 商品選項貨號: 4535304291626;", 'count': '1', 'price': '140', 'iso': '4535304291626'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜SANADA 強力枕頭曬架 曬枕頭 曬玩偶 -2色 粉色 紫色🌳 綠光森林 🌳; 商品選項名稱 (品):紫色; 價格: $ 55; 數量: 1; 商品選項貨號: 4973430015196 ;', 'count': '1', 'price': '55', 'iso': '4973430015196'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜SANADA 強力枕頭曬架 曬枕頭 曬玩偶 -2色 粉色 紫色🌳 綠光森林 🌳; 商品選項名稱 (品):粉色; 價格: $ 55; 數量: 1; 商品選項貨號: 4973430019736 ;', 'count': '1', 'price': '55', 'iso': '4973430019736'}]}, {'note': ' , ', 'orderdate': '2019-03-27 22:56', 'orderID': '19032722556BT8A', 'name': '陳*樺', 'tel': '886912***708', 'account': 'simple152w5', 'shipPrice': '60.00', 'ship': '全家', 'shipID': '09968614483', 'orderPrice': '900.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):💯㊣附發票🔜日本John's Blend 香氛片 白麝香 紅酒 茉莉 香片 衣櫥芳香 房間室內 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香; 價格: $ 48; 數量: 5; 商品選項貨號: 4535304275800;", 'count': '5', 'price': '48', 'iso': '4535304275800'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 (超取限制12瓶); 價格: $ 120; 數量: 5; 商品選項貨號: 4535304275893;", 'count': '5', 'price': '120', 'iso': '4535304275893'}]}, {'note': ' , ', 'orderdate': '2019-03-27 22:14', 'orderID': '190327221266BJM', 'name': '常*姐', 'tel': '886976***336', 'account': 'vivian0514', 'shipPrice': '60.00', 'ship': '全家', 'shipID': '09968614484', 'orderPrice': '359.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜 【INOMATA】冰箱迷你收納架2入組 🌳 綠光森林 🌳; 價格: $ 39; 數量: 1; 主商品貨號: 4905596035800;', 'count': '1', 'price': '39', 'iso': '4905596035800'}, {'prdText': '商品名稱 (品):💯㊣附發票 🔜 KAO花王 餐桌用除菌清潔噴劑-綠茶香300ml / 限定款米奇米妮 🌳 綠光森林 🌳; 價格: $ 130; 數量: 2; 主商品貨號: 4901301326850;', 'count': '2', 'price': '130', 'iso': '4901301326850'}]}, {'note': ' , ', 'orderdate': '2019-03-27 21:24', 'orderID': '190327212260EPJ', 'name': '林*玉', 'tel': '886975***656', 'account': 'ggyy345', 'shipPrice': '60.00', 'ship': '全家', 'shipID': '09968614482', 'orderPrice': '220.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜【SKATER】彈跳吸管寶特瓶蓋水壺套&附替換吸管 米奇 Kitty 🌳 綠光森林 🌳; 商品選項名稱 (品):HELLO KITTY; 價格: $ 160; 數量: 1; 商品選項貨號: 4973307250705;', 'count': '1', 'price': '160', 'iso': '4973307250705'}]}, {'note': ' , ', 'orderdate': '2019-03-27 15:52', 'orderID': '190327155261011', 'name': '吳*澄', 'tel': '886983***396', 'account': 'nico.....', 'shipPrice': '45.00', 'ship': '萊爾富', 'shipID': '9FS913528504', 'orderPrice': '1433.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': "商品名稱 (品):💯㊣附發票🔜日本John's Blend 香氛片 白麝香 紅酒 茉莉 香片 衣櫥芳香 房間室內 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香; 價格: $ 48; 數量: 1; 商品選項貨號: 4535304275800;", 'count': '1', 'price': '48', 'iso': '4535304275800'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜  John's Blend 香氛噴霧280ml 白麝香 蘋果梨 白茉莉 紅酒香 🌳 綠光森林 🌳; 商品選項名稱 (品):白茉莉 宅配限15個; 價格: $ 175; 數量: 2; 商品選項貨號: 4535304287520;", 'count': '2', 'price': '175', 'iso': '4535304287520'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白麝香 (超取限制12瓶); 價格: $ 120; 數量: 2; 商品選項貨號: 4535304275893;", 'count': '2', 'price': '120', 'iso': '4535304275893'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):白茉莉 (超取限制12瓶); 價格: $ 120; 數量: 1; 商品選項貨號: 4535304287681;", 'count': '1', 'price': '120', 'iso': '4535304287681'}, {'prdText': "商品名稱 (品):💯㊣附發票 🔜 正品日本 John's Blend 香氛膏/芳香膠 135g 房間室內芳香 🌳 綠光森林 🌳; 商品選項名稱 (品):麝香櫻花 (超取限制12瓶); 價格: $ 140; 數量: 2; 商品選項貨號: 4535304291626;", 'count': '2', 'price': '140', 'iso': '4535304291626'}, {'prdText': "商品名稱 (品):✨現貨快出🔜【John's Blend】香氛噴霧280ml 麝香櫻花 🌳 綠光森林 🌳; 商品選項名稱 (品):香氛噴霧280ml (超商限15瓶); 價格: $ 175; 數量: 2; 商品選項貨號: 4535304291619;", 'count': '2', 'price': '175', 'iso': '4535304291619'}]}, {'note': '盡快幫我出貨 謝謝 , ', 'orderdate': '2019-03-27 23:45', 'orderID': '19032723456HY01', 'name': '曾婕妮', 'tel': '886975375014', 'account': 'zeng0625', 'shipPrice': '0.00', 'ship': '賣家宅配', 'shipID': '', 'orderPrice': '480.00', 'discount': '0.00', 'credit': '信用卡', 'prds': [{'prdText': '商品名稱 (品):✨現貨快出🔜免運費【LIVEWELL】多功能手提式野餐桌 5色 日本製 🌳 綠光森林 🌳; 商品選項名稱 (品):抹茶色; 價格: $ 480; 數量: 1; 商品選項貨號: 4905310061382;', 'count': '1', 'price': '480', 'iso': '4905310061382'}]}, {'note': ' , 打統編42232299', 'orderdate': '2019-03-27 21:47', 'orderID': '1903272146636GC', 'name': '許甄齡', 'tel': '886926160301', 'account': 'hsuchenling', 'shipPrice': '80.00', 'ship': '賣家宅配', 'shipID': '', 'orderPrice': '1380.00', 'discount': '0.00', 'credit': '信用卡', 'prds': [{'prdText': '商品名稱 (品):限宅配 💯㊣附發票 🔜 日本 Living 玻璃梅森果汁桶3L 梅森桶 玻璃桶 飲料桶 🌳 綠光森林 🌳; 商品選項名稱 (品):玻璃梅森果汁桶3L; 價格: $ 650; 數量: 2; 商品選項貨號: 4546490706696;', 'count': '2', 'price': '650', 'iso': '4546490706696'}]}, {'note': '大樓有管理室可代收。 , ', 'orderdate': '2019-03-27 17:10', 'orderID': '190327163164R1M', 'name': '王柏堯', 'tel': '886910655227', 'account': 'abovewang', 'shipPrice': '80.00', 'ship': '賣家宅配', 'shipID': '', 'orderPrice': '1750.00', 'discount': '0.00', 'credit': '', 'prds': [{'prdText': '商品名稱 (品):📣限宅配 ✨現貨快出🔜【HIRO】珪藻土浴室吸水防滑墊 L 2.2kg 白色 🌳 綠光森林 🌳; 價格: $ 835; 數量: 2; 主商品貨號: 4562351019002;', 'count': '2', 'price': '835', 'iso': '4562351019002'}]}]
for i, v in a[0].items():
    print(i,v)

In [ ]:
#蝦皮庫存表整理 舊版
import xlrd
import xlwt
import re
import time
import requests
from bs4 import BeautifulSoup
import tkinter as tk
import tkinter.filedialog as filedialog
import os
#時間
def run():
    today_str = time.strftime("%Y",time.localtime())+"/"+str(int(time.strftime("%m",time.localtime())))+"/"+str(int(time.strftime("%d",time.localtime())))


    #ISO 資料
    url =  "https://docs.google.com/spreadsheets/d/19ZXwhENPrLmLURoKO4xXoCDahpyMG5wuU_8xsU74kyI/gviz/tq?tqx=out:html&tq=select%20B,C,D&gid=0"
    gotrq = requests.get(url)
    soup = BeautifulSoup(gotrq.text, "html.parser")
    tr = soup.select("tr")
    global isoAry
    isoAry = [] #獲取網站上的ISO 資料
    for r in tr:    
        try:
            isoAry.append([r.td.text,r.td.next_sibling.text,r.td.next_sibling.next_sibling.text])
        except:
            print('notd')

    #讀取XLS
    xlrd.Book.encoding = "utf8" #设置编码
    data = xlrd.open_workbook(filepath)
    table1 = data.sheet_by_index(0) #取第一张工作簿
    rows_count = table1.nrows #取总行数


    #商品資訊 陣列
    orderObj = []
    for i in range(1,rows_count): #迴圈
        orderObj.append({'note':"",'orderdate':"",'orderID':"",'name':"",'tel':"",'account':"",'shipPrice':"",'ship':"",'shipID':"",'orderPrice':"",'discount':"",'credit':"",'prds':[]})#新增新的物件
        orderObj[i-1]['orderID'] = table1.cell_value(i,0) 
        orderObj[i-1]['orderdate'] = table1.cell_value(i,5) 
        orderObj[i-1]['name'] = table1.cell_value(i,20) 
        orderObj[i-1]['tel'] = table1.cell_value(i,21) 
        orderObj[i-1]['account'] = table1.cell_value(i,3) 
        orderObj[i-1]['shipPrice'] = table1.cell_value(i,7) 
        orderObj[i-1]['ship'] = table1.cell_value(i,22) 
        orderObj[i-1]['shipID'] = table1.cell_value(i,28) 
        orderObj[i-1]['orderPrice'] = table1.cell_value(i,8) 
        orderObj[i-1]['discount'] = table1.cell_value(i,11) 
        orderObj[i-1]['note'] = table1.cell_value(i,31)+" , "+table1.cell_value(i,32)
        if table1.cell_value(i,25) == "信用卡":
            orderObj[i-1]['credit'] = table1.cell_value(i,25) 
        text = table1.cell_value(i,12) 
        if text != '商品資訊':
            text = text.replace("	","") #去除tab 大空格
            for ii in re.split("\[[0-9][0-9]*\]",text): #分割"[數字]"資料
                if ii: #如果ii有資料則輸出
                    orgtext = ii.replace("\n","").strip()
                    orderObj[i-1]['prds'].append({'prdText':orgtext,'count':prdinfo("數量:",orgtext),'price':prdinfo("價格: $",orgtext),'iso':prdinfo('商品選項貨號:',orgtext)+prdinfo('主商品貨號:',orgtext)}) #放入prd陣列中，-1是因為i是從第二列開始
            #折扣 新增一列折扣碼
            if float(orderObj[i-1]['discount']):
                orderObj[i-1]['prds'].append({'prdText':'折扣碼','count':"1",'price': float(orderObj[i-1]['discount'])*-1,'iso':"折扣碼"}) 


    #排序訂單
    orderObj.sort(key=order_time,reverse=True)
    orderObj.sort(key=order_ship)

    print(orderObj)
    #寫XLS
    nwb = xlwt.Workbook()# 新增EXCEL
    nsheet = nwb.add_sheet('google試算表格式')#新SHEET
    nsheet.write(0,0,'日期')
    nsheet.write(0,1,'訂單編號')
    nsheet.write(0,2,'姓名')
    nsheet.write(0,3,'電話')
    nsheet.write(0,4,'帳號')
    nsheet.write(0,5,'國際條碼')
    nsheet.write(0,6,'產品名稱')
    nsheet.write(0,7,'產品款式')
    nsheet.write(0,8,'數量')
    nsheet.write(0,9,'價格')
    nsheet.write(0,10,'總價')
    nsheet.write(0,11,'貨運')
    nsheet.write(0,12,'運費')
    nsheet.write(0,13,'發票')
    nsheet.write(0,16,'付款方式')
    nsheet.write(0,21,'折扣')
    nsheet.write(0,22,'備註')
    write_row = 1

    for i in orderObj:
        orderstart = 0
        for ii in i['prds']:
            orderstart += 1
            if orderstart == 1:
                nsheet.write(write_row,12,int(float(i['shipPrice'])))
                nsheet.write(write_row,11,i['ship'])
                nsheet.write(write_row,22,i['note'])
                nsheet.write(write_row,13,int(float(i['orderPrice'])-float(i['discount'])))#扣掉折扣 這是訂單上的折扣(正)，非商品資訊內折扣(負號)
                if i['credit'] == '信用卡':
                    nsheet.write(write_row,16,i['credit'])
                if float(i['discount']):
                    nsheet.write(write_row,21,int(float(i['discount'])))
            nsheet.write(write_row,0,today_str)
            nsheet.write(write_row,1,i['orderID'])
            nsheet.write(write_row,2,i['name']+i['shipID'][-3:])
            nsheet.write(write_row,3,i['tel'])
            nsheet.write(write_row,4,i['account'])
            
            #商品資料
            iso_data = isodetect(ii['iso'])
            if iso_data:
                nsheet.write(write_row,5,iso_data[0]) #ISO
                nsheet.write(write_row,6,iso_data[1]) #產品名
                nsheet.write(write_row,7,iso_data[2]) #款式
            else: #找不到ISO
                if ii['prdText'][6:ii['prdText'].find(" 價格:")] == '': #如果是折扣碼
                    print(write_row,ii['prdText'])
                    nsheet.write(write_row,6,ii['prdText'])
                else: #非折扣碼
                    nsheet.write(write_row,6,ii['prdText'][6:ii['prdText'].find(" 價格:")])
                    


            ct = ii['count']
            pe = ii['price']
            nsheet.write(write_row,8,ct)
            nsheet.write(write_row,9,int(float(pe)))
            nsheet.write(write_row,10,int(float(ct)*float(pe)))

            write_row += 1

    nwb.save('result.xls')



def order_time(ele): #排序的KEY
    return ele['orderdate']
def order_ship(ele):
    return ele['ship']

def prdinfo(findtitle,text): #分析商品資訊
    prdtext = text.find(findtitle)#找到數量的位置
    firstsplit = prdtext+len(findtitle)+text[prdtext+len(findtitle):len(text)].find(";") #找到數量之後第一個分號位置
    
    if prdtext == -1:
        return ""
    return text[prdtext+len(findtitle):firstsplit].strip().replace(",","")#需要的片段
def isodetect(iso):
    a = 0
    for i in isoAry:
        a += 1
        if i[0].find(iso) != -1:
            if i[0] == "": #如果找到的ISO碼是空的 則返回FALSE
                return False
            return i
        
    return False
def browser():
    fname = filedialog.askopenfilename(initialdir= os.getcwd(),filetypes = (("Excel","*.xls"),("all files","*.*")))
    global filepath
    filepath =  fname  # 返回文件全路径
    pathinput.delete('0',tk.END)
    pathinput.insert(0,filepath)
    
gui=tk.Tk()
gui.title("蝦皮出貨單")
gui.geometry("450x50")
pathinput = tk.Entry(width=50)
pathinput.pack(side="left")
btn2 = tk.Button(gui, text ="瀏覽", command = browser)
btn2.pack(side="left")
btn = tk.Button(gui, text ="執行", command = run)
btn.pack(side="left")
gui.mainloop()

